# Swissalti3d data acquisition

Script to download all relevant .tif files from the swissalti3d dataset for the Canton of Graubünden

In [4]:
import os
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm # Für den Ladebalken (pip install tqdm)

# Konfiguration
csv_file = '../data/ch.swisstopo.swissalti3d-cxnSHHnz.csv'       # Pfad zu deiner CSV von Swisstopo
download_dir = '../data/swissalti3d' # Ordner, wo die Tiffs landen sollen
max_parallel_downloads = 8             # Anzahl gleichzeitiger Downloads (je nach Internetleitung 4-16)

In [5]:
# 1. CSV einlesen
# header=None ist wichtig, da deine CSV keine Titelzeile hat
df = pd.read_csv(csv_file, header=None)

# Die erste Spalte (0) enthält die URLs. Leere Zeilen entfernen wir sicherheitshalber.
urls = df[0].dropna().tolist()

print(f"{len(urls)} Dateien zum Download gefunden.")

# 2. Download Funktion
def download_url(url):
    try:
        # Dateiname aus der URL holen (z.B. swissalti3d_..._5728.tif)
        filename = url.split('/')[-1]
        filepath = os.path.join(download_dir, filename)

        # Wenn Datei schon da ist und Größe > 0, überspringen (Resume-Funktion)
        if os.path.exists(filepath) and os.path.getsize(filepath) > 0:
            return # Datei existiert schon

        # Request senden
        r = requests.get(url.strip(), stream=True)
        if r.status_code == 200:
            with open(filepath, 'wb') as f:
                for chunk in r.iter_content(chunk_size=1024):
                    f.write(chunk)
    except Exception as e:
        print(f"Fehler bei {url}: {e}")

# 3. Paralleler Download starten
print("Starte Download... (Geduld, das sind viele Daten!)")
with ThreadPoolExecutor(max_workers=max_parallel_downloads) as executor:
    # tqdm erstellt den Ladebalken
    list(tqdm(executor.map(download_url, urls), total=len(urls)))

print("Fertig!")

7540 Dateien zum Download gefunden.
Starte Download... (Geduld, das sind viele Daten!)


100%|██████████| 7540/7540 [39:15<00:00,  3.20it/s]  

Fertig!
